# Visualize Inference Imagery with napari

In [ ]:
!pip install napari rioxarray -q

In [1]:
from pathlib import Path
from rioxarray.merge import merge_arrays

import rioxarray as rio
import napari
import numpy as np

In [13]:
# Pull in some files
bucket = "crop-mask-inference-tifs"
map_name = "Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1"
prefix = "batch_0/00000000"
!mkdir -p tifs
!gsutil -m cp -n "gs://{bucket}/{map_name}/{prefix}*" tifs

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000000-0000000000.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000000-0000000256.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000000-0000000512.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_m

Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000256-0000004864.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000256-0000005120.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000256-0000005376.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000256-0000005632.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000512-0000000000.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_la

Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000768-0000004352.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000768-0000004608.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000768-0000004864.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000768-0000005120.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_lat=12.25_min_lon=36.45_max_lat=14.91_max_lon=40.0_dates=2021-02-01_2022-02-01_all_v1/batch_0/00000000768-0000005376.tif...
Copying gs://crop-mask-inference-tifs/Ethiopia_Tigray_2021/min_la

In [14]:
ps = sorted(list(Path("tifs").glob("*.tif")))

In [15]:
next_row = 23
width = 4
start_index = 10

top_row = ps[start_index : width]
middle_row = ps[start_index + next_row : start_index + next_row + width]
bottom_row = ps[start_index + (next_row*2) : start_index + (next_row*2) + width]

In [16]:
da = merge_arrays([rio.open_rasterio(p, cache=False) for p in (top_row + middle_row + bottom_row)])

In [19]:
def normalize(arr):
    real_min = np.nanmin(arr)
    arr = np.nan_to_num(arr, nan=real_min)
    return (arr - arr.min()) / (arr.max() - arr.min())

def get_timesteps(band):
    band_timesteps = [da[i].data for i, b in enumerate(da.long_name) if band in b]
    return normalize(np.array(band_timesteps))


rgb = np.stack([get_timesteps(b) for b in ["B4", "B3", "B2"]], axis=-1)

vv = normalize(get_timesteps("VV"))
vh = normalize(get_timesteps("VH"))
sar = np.stack([vv, vh, vh-vv], axis=-1)

precipitation = get_timesteps("total_precipitation")
temperature = get_timesteps("temperature_2m")

b8 = get_timesteps("B8A")
b4 = get_timesteps("B4")
ndvi = (b8 - b4) / (b8 + b4)

elevation = normalize(da[-2].data)
slope = normalize(da[-1].data)

viewer = napari.view_image(rgb, contrast_limits=[0.0, 0.5])
viewer.add_image(sar)
viewer.add_image(temperature, colormap="red", opacity=0.3)
viewer.add_image(precipitation, colormap="blue", opacity=0.3)
viewer.add_image(ndvi, colormap="PiYG", opacity=0.3)
viewer.add_image(elevation, colormap="inferno", opacity=0.3)
viewer.add_image(slope, colormap="inferno", opacity=0.3)

<Image layer 'slope' at 0x15f497040>